In [12]:
import os 
import sys
import datetime
import csv 

import cirq
import qsimcirq
import openfermion
import numpy as np
Pi=3.1415


In [13]:
from functools import partial

## anzats

In [14]:
class AnzatsAFMHeisenberg():
    def __init__(self, length, gamma, beta):

        # initialize circuit
        circuit = cirq.Circuit()
        qubits = cirq.LineQubit.range(length)


        for i in range(int(length/2)):
            circuit.append(cirq.H(qubits[int(2*i)]))
            circuit.append(cirq.CNOT(qubits[int(2*i)], qubits[int(2*i+1)]))

        # add gamma circuit
        for index in range(gamma.size):
            # add gates on 2n and 2n+1
            for i in range(1, length, 2):
                circuit.append(
                    cirq.XX(qubits[i], qubits[(i+1)%length]) ** (-gamma[index]*2/Pi)
                    )
                circuit.append(
                    cirq.YY(qubits[i], qubits[(i+1)%length]) ** (-gamma[index]*2/Pi)
                    )
                circuit.append(
                    cirq.ZZ(qubits[i], qubits[(i+1)%length]) ** (-gamma[index]*2/Pi)
                    )
            # add gates on 2n-1 and 2n

            # add beta circuit
            for i in range(0, length, 2):
                circuit.append(
                    cirq.XX(qubits[i], qubits[(i+1)%length]) ** (-beta[index]*2/Pi)
                    )
                circuit.append(
                    cirq.YY(qubits[i], qubits[(i+1)%length]) ** (-beta[index]*2/Pi)
                    )
                circuit.append(
                    cirq.ZZ(qubits[i], qubits[(i+1)%length]) ** (-beta[index]*2/Pi)
                    )

        self.circuit = circuit
        self.qubits = qubits
        self.gamma = gamma
        self.beta = beta

class Anzats():
    def __init__(self, length, gamma, beta):

        # initialize circuit
        circuit = cirq.Circuit()
        qubits = cirq.LineQubit.range(length)

        for i in range(length):
            circuit.append(cirq.H(qubits[i]))

        # add gamma circuit
        for index in range(gamma.size):
            # add gates on 2n and 2n+1
            for i in range(1, length, 2):
                circuit.append(
                    cirq.ZZ(qubits[i], qubits[(i+1)%length]) ** (gamma[index]*2/Pi)
                    )
                # circuit.append(
                #     cirq.ZZPowGate(
                #         exponent=gamma[index]/Pi, global_shift=0.0).on(
                #             qubits[i], qubits[(i+1)%length]
                #     )
                # )
            # add gates on 2n-1 and 2n
            for i in range(0, length, 2):
                circuit.append(
                    cirq.ZZ(qubits[i], qubits[(i+1)%length]) ** (gamma[index]*2/Pi)
                    )
                # circuit.append(
                #     cirq.ZZPowGate(
                #         exponent=gamma[index]/Pi, global_shift=0.0).on(
                #             qubits[i], qubits[(i+1)%length]
                #         )
                # )

            # add beta circuit
            for i in range(length):
                circuit.append(
                    cirq.XPowGate(
                        exponent=beta[index]*2/Pi, global_shift=0.0).on(
                        qubits[i])
                )

        self.circuit = circuit
        self.qubits = qubits
        self.gamma = gamma
        self.beta = beta


## expectation

In [15]:
class AFMHeisenbergArgs():
    def __init__(self, length, qsim_option):
        self.length = length
        self.qsim_option = qsim_option


def get_expectation_afm_heisenberg(function_args, gamma, beta):
    anzats = AnzatsAFMHeisenberg(function_args.length, gamma, beta)
    circuit = anzats.circuit
    qubits = anzats.qubits
    simulator = qsimcirq.QSimSimulator(function_args.qsim_option)
    vector = simulator.simulate(circuit).state_vector()

    value = 0 + 0j
    for i in range(function_args.length):
        circuitX = anzats.circuit.copy()
        circuitY = anzats.circuit.copy()
        circuitZ = anzats.circuit.copy()

        circuitX.append(cirq.X(qubits[i]))
        circuitX.append(cirq.X(qubits[(i+1)%function_args.length]))
        vector2 = simulator.simulate(circuitX).state_vector()
        value += np.dot(vector2.conj(), vector)

        circuitY.append(cirq.Y(qubits[i]))
        circuitY.append(cirq.Y(qubits[(i+1)%function_args.length]))
        vector2 = simulator.simulate(circuitY).state_vector()
        value += np.dot(vector2.conj(), vector)

        circuitZ.append(cirq.Z(qubits[i]))
        circuitZ.append(cirq.Z(qubits[(i+1)%function_args.length]))
        vector2 = simulator.simulate(circuitZ).state_vector()
        value += np.dot(vector2.conj(), vector)
    return value

def get_expectation_critical_state(function_args, gamma, beta):
    anzats = Anzats(function_args.length, gamma, beta)
    circuit = anzats.circuit
    qubits = anzats.qubits
    vector = cirq.final_state_vector(circuit)

    value = 0 + 0j
    for i in range(function_args.length):
        circuit = anzats.circuit.copy()
        circuit.append(cirq.Z(qubits[i]))
        circuit.append(cirq.Z(qubits[(i+1)%function_args.length]))
        vector2 = cirq.final_state_vector(circuit)
        value -= np.dot(vector2.conj(), vector)
    for i in range(function_args.length):
        circuit = anzats.circuit.copy()
        # circuit.append(function_args.g * cirq.X(qubits[i]))
        circuit.append(cirq.XPowGate(exponent=function_args.g).on(qubits[i]))
        vector2 = cirq.final_state_vector(circuit)
        value -= np.dot(vector2.conj(), vector)
    return value

def get_expectation_ghz(length, gamma, beta):
    anzats = Anzats(length, gamma, beta)
    circuit = anzats.circuit
    qubits = anzats.qubits
    vector = cirq.final_state_vector(circuit)

    value = 0 + 0j
    for i in range(length):
        circuit = anzats.circuit.copy()
        circuit.append(cirq.Z(qubits[i]))
        circuit.append(cirq.Z(qubits[(i+1)%length]))
        vector2 = cirq.final_state_vector(circuit)
        value -= np.dot(vector2.conj(), vector)
    return value


def get_expectation_ghz_l4(gamma: np.array, beta: np.array):
    return get_expectation_ghz(4, gamma, beta)

def get_expectation_ghz_l8(gamma: np.array, beta: np.array):
    return get_expectation_ghz(8, gamma, beta)

def get_expectation_ghz_l10(gamma: np.array, beta: np.array):
    return get_expectation_ghz(10, gamma, beta)

def get_expectation_ghz_l12(gamma: np.array, beta: np.array):
    return get_expectation_ghz(12, gamma, beta)

def get_expectation_ghz_l14(gamma: np.array, beta: np.array):
    return get_expectation_ghz(14, gamma, beta)

def get_expectation_ghz_l16(gamma: np.array, beta: np.array):
    return get_expectation_ghz(16, gamma, beta)


## optimization

In [16]:

def get_gradient(function, gamma: np.array, beta: np.array, delta_gamma, delta_beta, iter):
    grad_gamma = np.zeros_like(gamma)
    grad_beta  = np.zeros_like(beta)
    gamma_edge = gamma
    beta_edge  = beta
    # initial gamma, beta?
    
    if not (gamma.size == beta.size):
        return 1

    for index in range(gamma.size):
        center = gamma[index]
        gamma_edge[index] = gamma[index] - delta_gamma
        e1 = function(gamma=gamma_edge, beta=beta)
        gamma_edge[index] = gamma[index] + delta_gamma
        e2 = function(gamma=gamma_edge, beta=beta)
        grad_gamma[index] = (e2.real-e1.real)/(2*delta_gamma)
        gamma[index] = center

        center = beta[index]
        beta_edge[index] = beta[index] - delta_beta
        e1 = function(gamma=gamma, beta=beta_edge)
        beta_edge[index] = beta[index] + delta_beta
        e2 = function(gamma=gamma, beta=beta_edge)
        grad_beta[index] = (e2.real-e1.real)/(2*delta_beta)
        beta[index] = center
    
    return grad_gamma, grad_beta


def optimize_by_gradient_descent(function, initial_gamma: np.array, initial_beta: np.array, alpha, delta_gamma, delta_beta, iteration, Figure=True,filepath=""):
    gamma, beta = initial_gamma, initial_beta

    textlines = []
    headline = ["iter", "energy"]
    for p in range(int(len(initial_gamma))):
        headline.append("gamma[{}]".format(p))
        headline.append("bata[{}]".format(p))
    print(headline)
    textlines.append(headline)

    for iter in range(int(iteration)):
        # it is complex for me to set get_gradient for two optical parameter_vector
        grad_gamma, grad_beta = get_gradient(function, gamma, beta, delta_gamma, delta_beta, iter)
        gamma -= alpha * grad_gamma
        beta  -= alpha * grad_beta
        energy = function(gamma=gamma, beta=beta)

        record = [iter, energy]
        for index in range(gamma.size):
            record.append(gamma[index])
            record.append(beta[index])
        textlines.append(record)
        print(record)
    
    if len(filepath)>0:
        with open(filepath, mode='a') as f:
            writer = csv.writer(f)
            for i, textline in enumerate(textlines):
                writer.writerow(textline)
                # f.write("{}\n".format(textline))

    return gamma, beta


## case: length 4, static gamma and beta

In [19]:
length = 4
g = 0.8
gamma = np.array([0.5, 0.4, 0.3, 0.2])
beta  = np.array([0.7, 0.5, 0.3, 0.1])
qsim_option = {'t': int(length/2), 'f':1}
function_args = AFMHeisenbergArgs(length, qsim_option)

anzats = AnzatsAFMHeisenberg(length, gamma, beta)
circuit = anzats.circuit
# print(circuit)

simulator = cirq.Simulator()
result = simulator.simulate(circuit, qubit_order=anzats.qubits)
vector = cirq.final_state_vector(circuit)
print(result)

norm = np.dot(vector.conjugate(), vector)
print("norm: {}".format(norm))

value = get_expectation_afm_heisenberg(function_args,gamma, beta)
print("expectation on afm: {}".format(value))

measurements: (no measurements)

qubits: (cirq.LineQubit(0), cirq.LineQubit(1), cirq.LineQubit(2), cirq.LineQubit(3))
output vector: (0.422+0.268j)|0000⟩ + (-0.074+0.026j)|0011⟩ + (0.16-0.073j)|0101⟩ + (0.337+0.316j)|0110⟩ + (0.337+0.316j)|1001⟩ + (0.16-0.073j)|1010⟩ + (-0.074+0.026j)|1100⟩ + (0.422+0.268j)|1111⟩

phase:
output vector: |⟩
norm: (0.9999998807907104+0j)
expectation on afm: (2.0217545330524445-8.568167686462402e-08j)


## case: length 8, static gamma and beta (paper value)

In [21]:
length = 8
gamma = np.array([0.2496, 0.4808, 0.5260, 0.4503])
beta  = np.array([0.6845, 0.6559, 0.6048, 0.3180])
qsim_option = {'t': int(length/2), 'f':1}
function_args = AFMHeisenbergArgs(length, qsim_option)

anzats = AnzatsAFMHeisenberg(length, gamma, beta)
circuit = anzats.circuit
# print(circuit)

simulator = cirq.Simulator()
result = simulator.simulate(circuit, qubit_order=anzats.qubits)
vector = cirq.final_state_vector(circuit)
print(result)

norm = np.dot(vector.conjugate(), vector)
print("norm: {}".format(norm))

value = get_expectation_afm_heisenberg(function_args,gamma, beta)
print("expectation on afm: {}".format(value))

measurements: (no measurements)

qubits: (cirq.LineQubit(0), cirq.LineQubit(1), cirq.LineQubit(2), cirq.LineQubit(3), cirq.LineQubit(4), cirq.LineQubit(5), cirq.LineQubit(6), cirq.LineQubit(7))
output vector: [ 0.23533559-0.08436342j  0.        +0.j          0.        +0.j
  0.15652958+0.02550919j  0.        +0.j          0.07459482-0.02911247j
 -0.0553033 -0.09889513j  0.        +0.j          0.        +0.j
  0.04454808+0.06785601j  0.07459487-0.0291125j   0.        +0.j
  0.15652958+0.02550911j  0.        +0.j          0.        +0.j
 -0.00298633-0.05241306j  0.        +0.j         -0.06902919-0.01421128j
  0.0094008 -0.00639707j  0.        +0.j          0.07459489-0.02911248j
  0.        +0.j          0.        +0.j          0.06400434-0.04483954j
 -0.05530327-0.09889513j  0.        +0.j          0.        +0.j
  0.01480664+0.00185153j  0.        +0.j          0.05658764+0.00704171j
  0.04340885-0.0141392j   0.        +0.j          0.        +0.j
  0.00940077-0.00639713j -0.0690292 

## case: length 4, optimize beta and gamma

In [22]:
length = 4
initial_gamma = np.array([0.1, 0.2])
initial_beta  = np.array([0.1, 0.2])
qsim_option = {'t': int(length/2), 'f':1}
function_args = AFMHeisenbergArgs(length, qsim_option)
iteration = 10
alpha = 0.1
delta_gamma = 0.001
delta_beta  = 0.001

gamma, beta = optimize_by_gradient_descent(partial(get_expectation_afm_heisenberg, function_args=function_args), initial_gamma, initial_beta, alpha, delta_gamma, delta_beta, iteration)

print(gamma, beta)

['iter', 'energy', 'gamma[0]', 'bata[0]', 'gamma[1]', 'bata[1]']
[0, (0.2187342755496502+2.60770320892334e-08j), 0.4807641565799713, -0.037451291084289545, 0.6397787153720855, 0.05053241252899171]
[1, (1.1900658756494522+1.1175870895385742e-08j), 0.701845234632492, -0.14771038442850112, 0.9032539695501327, -0.17429574877023696]
[2, (1.1598168462514877+2.9802322387695312e-08j), 0.9346103131771087, -0.41851924508810046, 1.4297220528125762, -0.3955221131443977]
[3, (0.05414157500490546+3.725290298461914e-09j), 1.3299258261919022, -0.7572293445467949, 1.4336108833551406, -0.8218324139714241]
[4, (1.5505704432725906+7.450580596923828e-09j), 1.229127759486437, -0.7614590624347329, 1.352195711620152, -0.7135519387200475]
[5, (2.1542739868164062-5.960464477539063e-08j), 1.645373070985079, -1.0877233395352959, 1.7157937305048108, -1.1610122026875616]
[6, (0.48428836464881897-4.470348358154297e-08j), 1.5739383943378926, -1.2830730924382805, 1.7561669366434216, -0.9601773323491216]
[7, (0.6008521

## case: length 8, optimize beta and gamma

In [ ]:
length = 8
p = 4
initial_gamma = np.array([0.5 for i in range(p)])
initial_beta  = np.array([0.5 for i in range(p)])
qsim_option = {'t': int(length/2), 'f':1}
function_args = AFMHeisenbergArgs(length, qsim_option)

iteration = 10
alpha = 0.01
delta_gamma = 0.001
delta_beta  = 0.001

# print("exact energy: ")
# print(-1.28*length)

gamma, beta = optimize_by_gradient_descent(partial(get_expectation_afm_heisenberg, function_args=function_args), initial_gamma, initial_beta, alpha, delta_gamma, delta_beta, iteration)

print(gamma, beta)

['iter', 'energy', 'gamma[0]', 'bata[0]', 'gamma[1]', 'bata[1]', 'gamma[2]', 'bata[2]', 'gamma[3]', 'bata[3]']
[0, (0.45555176585912704+3.259629011154175e-09j), 0.5416986271739006, 0.4962985888123512, 0.4306037649512291, 0.48373284935951233, 0.4931047856807709, 0.4953065440058708, 0.5061965733766556, 0.5205375999212265]
[1, (-0.6957846730947495-9.313225746154785e-10j), 0.5523277372121811, 0.4928741157054901, 0.3602413572371006, 0.4764210917055607, 0.5089507550001144, 0.4862055107951164, 0.4978474900126457, 0.5463928654789925]
[2, (-1.5354581736028194-2.2351741790771484e-08j), 0.548767700791359, 0.48182566836476326, 0.31602856889367104, 0.4759640172123909, 0.5408381782472134, 0.4723704718053341, 0.4850309081375599, 0.5820809789001942]


KeyboardInterrupt: 

## case: length 8, optimize beta and gamma b

In [ ]:
length = 8
p = 4
initial_gamma = np.array([0.5 for i in range(p)])
initial_beta  = np.array([0.5 for i in range(p)])
function_args = AFMHeisenbergArgs(length)

iteration = 20
alpha = 0.01
delta_gamma = 0.001
delta_beta  = 0.001

# print("exact energy: ")
# print(-1.28*length)

gamma, beta = optimize_by_gradient_descent(partial(get_expectation_afm_heisenberg, function_args=function_args), initial_gamma, initial_beta, alpha, delta_gamma, delta_beta, iteration)

print(gamma, beta)

['iter', 'energy', 'gamma[0]', 'bata[0]', 'gamma[1]', 'bata[1]', 'gamma[2]', 'bata[2]', 'gamma[3]', 'bata[3]']
[0, (0.45555176585912704-3.259629011154175e-09j), 0.5416986271739006, 0.4962985888123512, 0.4306037649512291, 0.48373284935951233, 0.4931047856807709, 0.4953065440058708, 0.5061965733766556, 0.5205375999212265]
[1, (-0.6957846730947495+9.313225746154785e-10j), 0.5523277372121811, 0.4928741157054901, 0.3602413572371006, 0.4764210917055607, 0.5089507550001144, 0.4862055107951164, 0.4978474900126457, 0.5463928654789925]
[2, (-1.5354581736028194+2.2351741790771484e-08j), 0.548767700791359, 0.48182566836476326, 0.31602856889367104, 0.4759640172123909, 0.5408381782472134, 0.4723704718053341, 0.4850309081375599, 0.5820809789001942]
[3, (-2.2546687135472894-1.30385160446167e-08j), 0.5527249556034803, 0.44852962903678417, 0.30325805954635143, 0.47364768758416176, 0.5729049043729901, 0.4652157351374626, 0.4812724534422159, 0.6168562425300479]
[4, (-2.910012688487768-5.587935447692871e-0

## case: length 16, static beta and gamma

In [ ]:
length = 16
p = 8
initial_gamma = np.array([0.5 for i in range(p)])
initial_beta  = np.array([0.5 for i in range(p)])
function_args = AFMHeisenbergArgs(length)

iteration = 10
alpha = 0.01
delta_gamma = 0.001
delta_beta  = 0.001

# print("exact energy: ")
# print(-1.28*length)

value = get_expectation_afm_heisenberg(function_args,gamma, beta)
print("expectation on afm: {}".format(value))

expectation on afm: (1.8219750868156552-1.1525116860866547e-08j)


## playground

In [ ]:
L = 10
circuit = cirq.Circuit()
qubits = cirq.LineQubit.range(L)

for i in range(1, L, 2):
    circuit.append(
        cirq.ZZPowGate(
            exponent=0.5, global_shift=0.0).on(
                qubits[i-1], qubits[i]
        )
    )


for i in range(2, L, 2):
    circuit.append(
        cirq.ZZPowGate(
            exponent=0.5, global_shift=0.0).on(
                qubits[i-1], qubits[i]
            )
    )


for i in range(L):
    circuit.append(
        cirq.XPowGate(
            exponent=0.5, global_shift=0.0).on(
            qubits[i])
    )


# for i in range(1, L, 2):
#     circuit.append(cirq.Z(qubits[i-1]))
#     circuit.append(cirq.Z(qubits[i]))


# for i in range(2, L, 2):
#     circuit.append(cirq.Y(qubits[i-1]))
#     circuit.append(cirq.Y(qubits[i]))


# for i in range(L):
#     circuit.append(cirq.X(qubits[i]))



print(circuit)


0: ───ZZ───────X^0.5────────────
      │
1: ───ZZ^0.5───ZZ───────X^0.5───
               │
2: ───ZZ───────ZZ^0.5───X^0.5───
      │
3: ───ZZ^0.5───ZZ───────X^0.5───
               │
4: ───ZZ───────ZZ^0.5───X^0.5───
      │
5: ───ZZ^0.5───ZZ───────X^0.5───
               │
6: ───ZZ───────ZZ^0.5───X^0.5───
      │
7: ───ZZ^0.5───ZZ───────X^0.5───
               │
8: ───ZZ───────ZZ^0.5───X^0.5───
      │
9: ───ZZ^0.5───X^0.5────────────


In [ ]:
value = 0 + 0j

# circuit2 = circuit
# anzats2 = Anzats(length, gamma, beta)
# circuit2 = anzats2.circuit
# qubits = anzats2.qubits
# circuit2.append(
#         cirq.ZZPowGate(
#             exponent=1, global_shift=0.0).on(
#                 qubits[1], qubits[2]
#         ))
# print(circuit2)
# result2 = simulator.simulate(circuit2, qubit_order=anzats.qubits)
# vector2 = cirq.final_state_vector(circuit2)
# value -= np.dot(vector2.conjugate(), vector)

# print(value)

# # for i in range(length-1):
#     circuit2 = circuit.append(cirq.ZZ.on(
#         qubits[i], qubits[i+1]
#     ))
#     result2 = simulator.simulate(circuit2, qubit_order=anzats.qubits)
#     vector2 = cirq.final_state_vector(circuit2)
#     value -= np.dot(vector2.conjugate(), vector)
# else:
#     circuit2 = circuit.append(cirq.ZZ.on(
#         qubits[length-1], qubits[0]
#     ))
#     result2 = simulator.simulate(circuit2, qubit_order=anzats.qubits)
#     vector2 = cirq.final_state_vector(circuit2)
#     value -= np.dot(vector2.conjugate(), vector)

In [ ]:
a = np.array([1,3])
a[1]

3

In [ ]:
print(vector)

[0.4591058 -4.49337065e-03j 0.12919496-1.25825405e-03j
 0.12919495-1.25825033e-03j 0.07737847-7.35387206e-04j
 0.12919495-1.25830155e-03j 0.04038734-3.95298004e-04j
 0.07737847-7.35415146e-04j 0.06066927-5.82884997e-04j
 0.12919495-1.25823729e-03j 0.03783051-3.70055437e-04j
 0.04038734-3.95270996e-04j 0.02739706-2.70117074e-04j
 0.07737848-7.35405833e-04j 0.02739704-2.70104036e-04j
 0.06066925-5.82892448e-04j 0.05631588-5.69688156e-04j
 0.12919493-1.25823729e-03j 0.03746548-3.55772674e-04j
 0.03783053-3.70066147e-04j 0.0245581 -2.27299519e-04j
 0.04038735-3.95267271e-04j 0.01372327-1.31545588e-04j
 0.02739703-2.70092394e-04j 0.02465377-2.33075581e-04j
 0.0773785 -7.35389534e-04j 0.02455811-2.27298588e-04j
 0.02739705-2.70094723e-04j 0.02209694-2.07859091e-04j
 0.06066927-5.82899433e-04j 0.02465375-2.33095139e-04j
 0.05631589-5.69676980e-04j 0.06066924-5.82894310e-04j
 0.12919495-1.25824474e-03j 0.03783055-3.70055437e-04j
 0.03746548-3.55740078e-04j 0.0245581 -2.27305107e-04j
 0.0378305

In [ ]:
def function_norm(gamma, beta):
    return np.linalg.norm(gamma)**2 + np.linalg.norm(beta)**2

p = 1
initial_gamma = np.array([-3.0 for i in range(p)])
initial_beta  = np.array([4.0 for i in range(p)])

alpha = 0.1
delta_gamma = 0.0001
delta_beta = 0.0001
iteration = 100

final_gamma, final_beta = optimize_by_gradient_descent(function_norm, initial_gamma, initial_beta, alpha, delta_gamma, delta_beta, iteration, Figure=True)

print(final_gamma, final_beta)

['iter', 'energy', 'gamma[0]', 'bata[0]']
[0, 20.250009000048443, -2.6999950000002855, 3.6000049999995696]
[1, 16.402515390162296, -2.4299904999994766, 3.2400094999975835]
[2, 13.286044756250423, -2.186986449998926, 2.9160135499979276]
[3, 10.761702813850011, -1.9682828049986085, 2.6244171949971715]
[4, 8.71698518447511, -1.771449524498145, 2.3619804754972904]
[5, 7.060763314251251, -1.5942995720487048, 2.1257874279467757]
[6, 5.71922306798135, -1.4348646148437645, 1.9132136851514012]
[7, 4.632574990255437, -1.2913731533593626, 1.721897316635964]
[8, 3.7523896168740816, -1.1622308380235786, 1.5497125849721591]
[9, 3.039439077053899, -1.0460027542212842, 1.3947463264749569]
[10, 2.461948791156269, -0.9413974787991748, 1.2552766938276303]
[11, 1.9941813457998523, -0.8472527309192319, 1.1297540244450364]
[12, 1.6152894326596015, -0.7625224578273722, 1.0167836220005242]
[13, 1.3083867288548008, -0.6862652120446207, 0.9151102598005076]
[14, 1.0597953100277273, -0.617633690840111, 0.82360423

In [ ]:
def function_norm(gamma, beta):
    return np.linalg.norm(gamma)**2 + np.linalg.norm(beta)**2

initial_gamma = np.array([3.0, 2.0])
initial_beta = np.array([4.0, 5.0])

final_gamma, final_beta = optimize_by_gradient_descent(
    function=function_norm,
    initial_gamma=initial_gamma,
    initial_beta=initial_beta,
    alpha=0.1,
    delta_gamma=0.001,
    delta_beta=0.001,
    iteration=100,
    Figure=True)
print(final_gamma, final_beta)
print(function_norm(gamma, beta))

['iter', 'energy', 'gamma[0]', 'bata[0]', 'gamma[1]', 'bata[1]']
[0, 43.741260009996445, 2.7000499999998624, 3.6000500000000955, 1.8000499999999846, 4.500049999999618]
[1, 35.43155463610003, 2.4300949999999517, 3.2400950000003945, 1.6200949999998642, 4.0500949999997715]
[2, 28.700579899441102, 2.1871354999997124, 2.9161355000003795, 1.4581354999997558, 3.6451354999999808]
[3, 23.248388317329418, 1.9684719499996035, 2.6245719500004725, 1.3123719499998003, 3.2806719500002757]
[4, 18.83202129493629, 1.7716747549995944, 2.3621647550002365, 1.1811847549998404, 2.9526547549999904]
[5, 15.254681350009095, 1.5945572794996394, 2.1259982795000596, 1.0631162794997522, 2.657439279499947]
[6, 12.35696160350955, 1.4351515515496267, 1.9134484515502557, 0.9568546515497083, 2.391745351550174]
[7, 10.00974165684386, 1.2916863963946952, 1.7221536063953344, 0.8612191863947665, 2.1526208163951743]
[8, 8.108433243244505, 1.162567756755248, 1.5499882457558698, 0.7751472677553366, 1.9374087347556923]
[9, 6.56

In [ ]:
length = 6
gamma = np.array([0.1, 0.1])
beta = np.array([0.1, 0.1])

anzats = Anzats(length, gamma, beta)
circuit = anzats.circuit
qubits = anzats.qubits
vector = cirq.final_state_vector(circuit)

ham_nd = np.zeros((length, length), dtype=np.complex128)

print(cirq.Z(qubits[-1]))
for i in range(length-1):
    circuit_ZiZj = cirq.Circuit()
    circuit_ZiZj.append(cirq.Z(qubits[i]))
    circuit_ZiZj.append(cirq.Z(qubits[i+1]))
    ham_nd += cirq.unitary(circuit_ZiZj)
else:
    circuit_ZiZj = cirq.Circuit()
    circuit_ZiZj.append(cirq.Z(qubits[length-1]))
    circuit_ZiZj.append(cirq.Z(qubits[0]))
    ham_nd += cirq.unitary(circuit_ZiZj)
    
print(ham_nd)


# def get_expectation_ZiZj(length, beta, gamma, qubit_ZZ_on_i, qubit_ZZ_on_j, print_circuit=False):
#     """
#     controls of append on circuit are accumulated.
#     we have to make circuits on each hamiltonian terms.
#     """
#     anzats = Anzats(length, gamma, beta)
#     circuit = anzats.circuit
#     qubits = anzats.qubits
#     vector = cirq.final_state_vector(circuit)

#     anzats = Anzats(length, gamma, beta)
#     circuit = anzats.circuit
#     qubits = anzats.qubits
#     circuit.append(cirq.Z(qubits[qubit_ZZ_on_i]))
#     circuit.append(cirq.Z(qubits[qubit_ZZ_on_j]))
#     if print_circuit:
#         print(circuit)
#     simulator = cirq.Simulator()
#     # result2 = simulator.simulate(circuit, qubit_order=qubits)
#     vector2 = cirq.final_state_vector(circuit)
#     return np.dot(vector2.conjugate(), vector)


# def get_expectation_ghz_l4(gamma: np.array, beta: np.array):
#     length = 4
#     value = 0 + 0j
#     for i in range(length-1):
#         value -= get_expectation_ZiZj(length, gamma, beta, i, i+1)
#     else:
#         value -= get_expectation_ZiZj(length, gamma, beta, length-1, 0)
#     return value


# def get_expectation_ghz_l8(gamma: np.array, beta: np.array):
#     length = 8
#     value = 0 + 0j
#     for i in range(length-1):
#         value -= get_expectation_ZiZj(length, gamma, beta, i, i+1)
#     else:
#         value -= get_expectation_ZiZj(length, gamma, beta, length-1, 0)
#     return value


Z(q(5))


ValueError: operands could not be broadcast together with shapes (6,6) (4,4) (6,6) 